# Collecting data from fbref

We will start by collecting the shots data from fbref 2017-18 onwards (since that has the advanced stats like xG) for the top 5 leagues. First let's import the necessary packages.

In [1]:
%load_ext autoreload
%autoreload 2
from scipy.stats import beta
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd 
import numpy as np
import matplotlib.cm as cm

We will combine the player name and birth year to create unique combinations, drop goalkeepers

In [2]:
urls = ['https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2020-2021/shooting/players/2020-2021-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2019-2020/shooting/players/2019-2020-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2018-2019/shooting/players/2018-2019-Big-5-European-Leagues-Stats',
       'https://fbref.com/en/comps/Big5/2017-2018/shooting/players/2017-2018-Big-5-European-Leagues-Stats']

dfs = []
for url in urls:
    df = pd.read_html(url)[0]
    df.columns = [c[1] if 'Unnamed' in c[0] else c[0]+'_'+c[1] for c in df.columns]
    df = df[['Player','Pos','Born','90s','Standard_Gls','Standard_Sh','Standard_PK','Expected_npxG']]
    df = df[(df.Player != "Player") & (df.Pos.notna())]
    df = df[~df.Pos.str.contains('GK')].reset_index(drop=True)
    df['Player'] = df['Player'] + ' (' + df['Born'] + ')'
    for cols in ['90s','Standard_Gls','Standard_Sh','Standard_PK','Expected_npxG']:
        df[cols] = df[cols].astype(float)
    df.fillna(value=0.0, inplace=True)
    dfs.append(df)
    
df = pd.concat(dfs, ignore_index=True)

In [3]:
df.head()

,Player,Pos,Born,90s,Standard_Gls,Standard_Sh,Standard_PK,Expected_npxG
0,Max Aarons (2000),DF,2000,18.0,0.0,9.0,0.0,0.6
1,Yunis Abdelhamid (1987),DF,1987,18.0,1.0,14.0,0.0,1.0
2,Salis Abdul Samed (2000),MF,2000,15.7,0.0,6.0,0.0,0.4
3,Laurent Abergel (1993),MF,1993,17.6,0.0,16.0,0.0,1.6
4,Charles Abi (2000),FW,2000,0.5,0.0,0.0,0.0,0.0


We will use the unique combinations (hopefully) of name + birth year to groupby and get total shots, total non-penalty goals, total non-penalty xG, Age, total 90s played etc.

In [9]:
gdf = df.groupby('Player').sum().reset_index()
gdf.columns = ['Player','90s','Goals','Shots','PKs','npxG'] 
gdf['npG'] = gdf['Goals'] - gdf['PKs']
gdf = gdf[gdf.Player!=0.0].reset_index(drop=True)
gdf['Born'] = [float(gdf['Player'][i].split('(')[1].split(')')[0]) for i in range(len(gdf))]
gdf['Age'] = 2021 - gdf['Born']
gdf.to_csv('fbrefshootingdata.csv', index=False, encoding='utf-8-sig')

In [10]:
gdf

,Player,90s,Goals,Shots,PKs,npxG,npG,Born,Age
0,Aaron Connolly (2000),24.5,5.0,63.0,0.0,7.2,5.0,2000.0,21.0
1,Aaron Cresswell (1989),130.0,5.0,78.0,0.0,3.9,5.0,1989.0,32.0
2,Aaron Hickey (2002),24.5,4.0,18.0,0.0,0.7,4.0,2002.0,19.0
3,Aaron Hunt (1986),23.1,3.0,27.0,1.0,2.1,2.0,1986.0,35.0
4,Aaron Lennon (1987),43.2,2.0,23.0,0.0,1.8,2.0,1987.0,34.0
...,...,...,...,...,...,...,...,...,...
4702,Łukasz Teodorczyk (1991),6.9,1.0,16.0,0.0,1.4,1.0,1991.0,30.0
4703,Šime Vrsaljko (1992),39.1,1.0,8.0,0.0,1.0,1.0,1992.0,29.0
4704,Žan Celar (1999),0.1,0.0,0.0,0.0,0.0,0.0,1999.0,22.0
4705,Žan Majer (1992),18.5,1.0,20.0,0.0,2.3,1.0,1992.0,29.0
